In [12]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Tudu2U0BJtC_WXHTHg1wUJa7N2H_oz2E6NqDLZPO6SoA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'flightdelayprediction-donotdelete-pr-1zaiceedkan8es'
object_key = 'Processed_data15.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()


,year,month,day,carrier,origin,dest,delayed
0,2013,1,1,UA,EWR,IAH,0
1,2013,1,1,UA,LGA,IAH,1
2,2013,1,1,AA,JFK,MIA,1
3,2013,1,1,B6,JFK,BQN,0
4,2013,1,1,DL,LGA,ATL,0


In [13]:
dataset.describe()

,year,month,day,delayed
count,327346.0,327346.000000,327346.000000,327346.000000
mean,2013.0,6.564803,15.740825,0.279594
std,0.0,3.413444,8.777376,0.448800
min,2013.0,1.000000,1.000000,0.000000
25%,2013.0,4.000000,8.000000,0.000000
50%,2013.0,7.000000,16.000000,0.000000
75%,2013.0,10.000000,23.000000,1.000000
max,2013.0,12.000000,31.000000,1.000000


In [14]:
# Import libraries 
import pandas as pd 
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function

# Import dataset 
df =dataset

# Label Encoding
le_carrier = LabelEncoder()
df['carrier'] = le_carrier.fit_transform(df['carrier'])

le_dest = LabelEncoder()
df['dest'] = le_dest.fit_transform(df['dest'])

le_origin = LabelEncoder()
df['origin'] = le_origin.fit_transform(df['origin'])

# Converting Pandas DataFrame into a Numpy array
X = df.iloc[:, 0:6].values # from column(years) to column(distance)
y = df['delayed'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=18) # 70% training and 30% test
# For 75% Train and 25% test use Random state =809

#Create a Random Forest Classifier
clf = RandomForestClassifier(random_state=18)
clf.fit(X_train, y_train)

# Saving model to disk
pickle.dump(clf,open('model.pkl','wb'))

model = pickle.load(open('model.pkl','rb'))


In [27]:
print(X_train)


[[2013    4    6    1    1   11]
 [2013    4    8    1    1   49]
 [2013   11   25    9    2   60]
 ...
 [2013    4   22    5    0   24]
 [2013    7   23    4    1   92]
 [2013   10   10   12    2   28]]


In [16]:
apikey = 'rwsCVupJNjytOKgOkhLD3zUweTkAKg7329eJPSqU3_F4'
region = 'us-south'

cred = {
    'apikey': apikey,
    'url': 'https://'+ region+'.ml.cloud.ibm.com'
}
cred

{'apikey': 'rwsCVupJNjytOKgOkhLD3zUweTkAKg7329eJPSqU3_F4',
 'url': 'https://us-south.ml.cloud.ibm.com'}

In [17]:
space_id = 'c96a68f6-795c-4b44-be3b-22750c161b77'

In [18]:
# !pip install -U ibm-watson-machine-learning

from ibm_watson_machine_learning import APIClient 

client = APIClient(cred)

#Set the space that u have created
client.set.default_space(space_id)


print('space created\n',client.version)
client.spaces.list(limit=10)

space created
 1.0.257
------------------------------------  --------  ------------------------
ID                                    NAME      CREATED
c96a68f6-795c-4b44-be3b-22750c161b77  Akashram  2022-11-19T17:39:24.261Z
------------------------------------  --------  ------------------------


In [25]:
#find how many deployments are there
# client.spaces.list(limit=10)
soft_id = client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

meta_data  = {
    client.repository.ModelMetaNames.NAME : 'Random Forest model for Flight delay prediction',
    client.repository.ModelMetaNames.TYPE : 'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : soft_id
}

meta_data

{'name': 'Random Forest model for Flight delay prediction',
 'type': 'scikit-learn_1.0',
 'software_spec': '12b83a17-24d8-5082-900f-0ab31fbfd3cb'}

In [26]:
publish_model = client.repository.store_model(
    model =clf,
    meta_props= meta_data
)
print("Model Published in cloud Successfully!!!!")

Model Published in cloud Successfully!!!!
